Import

In [3]:
import pandas as pd
import datetime as dt
import numpy as np
import ast
import re
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import matplotlib.pyplot as plt

Fonction

In [4]:
# Fonction qui transforme les str en liste dans les df
def transfo_liste(x):
  if isinstance(x, str):
    return ast.literal_eval(x)
  else:
    return x
# Fonction qui nettoie les listes (supprimer les éléments vides et les doublons)
def clean_country_list(x):
    if isinstance(x, list):
        return list(set(pays for pays in x if pays))
    return x

# Fonction qui récupère les infos d'un film en web scrapping
def web(id):
    navigator = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1)'
    url_base_title = 'https://www.imdb.com/fr/title/'
    url_finale_title = f'{url_base_title}{id}'  

    html_title = requests.get(url_finale_title, headers={'User-Agent': navigator})
    html_title2 = html_title.content
    soup_title = BeautifulSoup(html_title2, 'html.parser')

    titre = None
    balise_titre = soup_title.find('span', class_='hero__primary-text')
    if balise_titre:
        titre = balise_titre.get_text().strip()

    description = None
    balise_description = soup_title.find('span', attrs={'data-testid': 'plot-xl'})
    if balise_description:
        description = balise_description.get_text().strip()

    realisateur = None
    balise_realisateur = soup_title.find('a', class_='ipc-metadata-list-item__list-content-item')
    if balise_realisateur:
        realisateur = balise_realisateur.get_text().strip()

    acteurs = []
    acteurs_balises = soup_title.find_all('a', attrs={'data-testid': 'title-cast-item__actor'}, limit=5)
    
    for acteur_balise in acteurs_balises:
        acteur_nom = acteur_balise.get_text().strip()
        acteurs.append(acteur_nom)

    dict = {}
    dict['Titre'] = titre
    dict['Description'] = description
    dict['Realisateur'] = realisateur
    dict['Acteurs'] = acteurs

    return dict


Chargement des tables et merge

In [5]:
# Import de la BD IMBD des notes
df_title_ratings = pd.read_csv('BD/title.ratings.tsv.gz', compression = 'gzip', sep = '\t', na_values= ['\\N'])

In [6]:
df_title_basics = pd.read_csv('BD/title.basics.tsv.gz', compression = 'gzip', sep = '\t', na_values= ['\\N'])
df_title_basics = df_title_basics[df_title_basics['titleType'] == 'movie']
df_title_basics = df_title_basics.drop(['titleType', 'endYear'], axis = 1)
df_title_basics['genres'] = df_title_basics['genres'].apply(lambda x: x.split(',') if isinstance(x, str) else [])

C:\Users\lepor\AppData\Local\Temp\ipykernel_21820\3695035267.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_title_basics = pd.read_csv('BD/title.basics.tsv.gz', compression = 'gzip', sep = '\t', na_values= ['\\N'])


In [7]:
df_merge1 = pd.merge(df_title_ratings,
                     df_title_basics,
                     how = 'right',
                     on = "tconst")

In [8]:
df_title_akas = pd.read_csv('BD/title.akas.tsv.gz', compression = 'gzip', sep = '\t', na_values= ['\\N'])
df_title_akas2 = pd.merge(df_merge1,
                         df_title_akas,
                         how = 'left',
                         left_on = "tconst",
                         right_on = "titleId")
df_title_akas3 = df_title_akas2.groupby('titleId').agg({
    'region': lambda x: list(x.dropna()),
    'language': lambda x: list(x.dropna())
}).reset_index()
df_title_akas3
df_merge2 = pd.merge(df_merge1,
                     df_title_akas3,
                     how = 'left',
                     left_on = "tconst",
                     right_on = "titleId")
df_merge2 = df_merge2.drop(['titleId'], axis = 1)

C:\Users\lepor\AppData\Local\Temp\ipykernel_21820\3813225811.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df_title_akas = pd.read_csv('BD/title.akas.tsv.gz', compression = 'gzip', sep = '\t', na_values= ['\\N'])


In [9]:
# Import de la BD TMBD
df_tmbd_full = pd.read_csv('BD/tmdb_full.csv', na_values= ['\\N'])
df_tmbd_full = df_tmbd_full.drop(['backdrop_path', 'budget', 'homepage', 'id', 'overview', 'poster_path', 'revenue', 'tagline', 'video', 'production_companies_country'], axis = 1) # suppression des colonnes inutiles

# Uniformisation des informations
df_tmbd_full['original_language'] = df_tmbd_full['original_language'].str.upper()
df_tmbd_full['spoken_languages'] = df_tmbd_full['spoken_languages'].str.upper()

# Nettoyage des listes
columns_to_transform = ['genres', 'production_countries', 'spoken_languages', 'production_companies_name']
for column in columns_to_transform:
    df_tmbd_full[column] = df_tmbd_full[column].apply(transfo_liste)
    df_tmbd_full[column] = df_tmbd_full[column].apply(clean_country_list)


C:\Users\lepor\AppData\Local\Temp\ipykernel_21820\412715110.py:2: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  df_tmbd_full = pd.read_csv('BD/tmdb_full.csv', na_values= ['\\N'])


In [10]:
df_film = pd.merge(df_merge2,
                   df_tmbd_full,
                   how = 'outer',
                   left_on = "tconst",
                   right_on = "imdb_id")

In [11]:
# Fusion des colonnes id
df_film['tconst'] = df_film.apply(lambda row: row['imdb_id'] if pd.notna(row['imdb_id']) else row['tconst'], axis=1)
df_film = df_film.drop(['imdb_id'], axis = 1)

# Fusion des colonnes de note 
df_film = df_film.applymap(lambda x: np.nan if x == 0 else x)
df_film[['vote_count', 'numVotes']] = df_film[['vote_count', 'numVotes']].fillna(0)
df_film[['vote_average', 'averageRating']] = df_film[['vote_average', 'averageRating']].fillna(0)
df_film['nb_de_votes'] = df_film['vote_count'] + df_film['numVotes']
df_film['moyenne_vote'] = ((df_film['vote_average'] * df_film['vote_count'] +df_film['averageRating'] * df_film['numVotes']) / df_film['nb_de_votes'])
df_film['moyenne_vote'] = df_film['moyenne_vote'].fillna(0)
df_film = df_film.drop(['vote_count', 'numVotes', 'vote_average', 'averageRating'], axis = 1)

C:\Users\lepor\AppData\Local\Temp\ipykernel_21820\4220460197.py:6: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_film = df_film.applymap(lambda x: np.nan if x == 0 else x)


In [12]:
df_film = df_film.loc[df_film['adult'] != True]
df_film = df_film.loc[df_film['isAdult'] != 1]
df_film = df_film.drop(['adult', 'isAdult'], axis = 1)

In [13]:
df_film['startYear'] = pd.to_numeric(df_film['startYear'], errors='coerce')
df_film['release_date'] = pd.to_datetime(df_film['release_date'], errors='coerce')
df_film['Date'] = df_film['startYear'].fillna(df_film['release_date'].dt.year)
df_film = df_film.drop(['release_date', 'startYear'], axis = 1)

In [14]:
df_film['runtimeMinutes'] = pd.to_numeric(df_film['runtimeMinutes'], errors='coerce')
df_film['runtime'] = pd.to_numeric(df_film['runtime'], errors='coerce')
df_film['Duree'] = df_film['runtimeMinutes'].fillna(df_film['runtime'])
df_film = df_film.drop(['runtimeMinutes', 'runtime'], axis = 1)


In [15]:
df_film['genres_x'] = df_film['genres_x'].apply(lambda x: x if isinstance(x, list) else [])
df_film['genres_y'] = df_film['genres_y'].apply(lambda x: x if isinstance(x, list) else [])
df_film['Genres'] = df_film['genres_x'] + df_film['genres_y']
df_film['Genres'] = df_film['Genres'].apply(lambda x: list(set(x)))
df_film = df_film.drop(['genres_x', 'genres_y'], axis = 1)

In [16]:
df_film['region'] = df_film['region'].apply(lambda x: x if isinstance(x, list) else [])
df_film['production_countries'] = df_film['production_countries'].apply(lambda x: x if isinstance(x, list) else [])
df_film['Region'] = df_film['region'] + df_film['production_countries']
df_film['Region'] = df_film['Region'].apply(lambda x: list(set(x)))
df_film = df_film.drop(['region', 'production_countries'], axis = 1)

In [17]:
df_film['original_language'] = df_film['original_language'].apply(lambda x: [str(x).upper()] if pd.notnull(x) else [])
df_film['language'] = df_film['language'].apply(lambda x: [str(i).upper() for i in x] if isinstance(x, list) else [])
df_film['spoken_languages'] = df_film['spoken_languages'].apply(lambda x: [str(i).upper() for i in x] if isinstance(x, list) else [])
df_film['Language'] = df_film['original_language'] + df_film['language'] + df_film['spoken_languages']
df_film['Language'] = df_film['Language'].apply(lambda x: list(set(x)))
df_film = df_film.drop(['original_language', 'language', 'spoken_languages'], axis = 1)


In [18]:
df_film['Titre'] = df_film['title'].combine_first(df_film['primaryTitle']).combine_first(df_film['original_title']).combine_first(df_film['originalTitle'])
df_film = df_film.drop(['title', 'primaryTitle', 'original_title', 'originalTitle'], axis = 1)

In [19]:
df_film = df_film[df_film['Language'].apply(lambda x: "FR" in x if isinstance(x, list) else False) |
                 df_film['Region'].apply(lambda x: "FR" in x if isinstance(x, list) else False)]


In [20]:
df_film = df_film[df_film['Date'] >= 1905]
df_film = df_film[(df_film['Duree'] >= 45) & (df_film['Duree'] <= 240)]
df_film = df_film[df_film['nb_de_votes'] >= 100]

In [21]:
df_film = df_film[df_film['popularity'].notna()]

In [22]:
df_film['Saga'] = df_film['Titre'].str[:8].str.replace(" ", "").str.upper()

In [23]:
df_film = df_film.drop(['status'], axis = 1)

In [24]:
df_film

,tconst,popularity,production_companies_name,nb_de_votes,moyenne_vote,Date,Duree,Genres,Region,Language,Titre,Saga
687,tt0002130,5.387,[Milano Films],3773.0,6.981394,1911.0,71.0,"[Adventure, Horror, Fantasy, Drama]","[MX, FR, DK, PT, RU, GB, US, AR, IT, CA, BR, A...","[XX, SV, IT, EN, JA]",L’Inferno,L’INFERN
715,tt0002199,1.162,[Kalem Company],679.0,5.786009,1912.0,71.0,"[Biography, Drama]","[SE, FR, US, RU, IE, BR, ES]","[EN, XX]",From the Manger to the Cross,FROMTHE
784,tt0002423,1.841,[Projektions-AG Union (PAGU)],1055.0,6.592796,1919.0,113.0,"[Romance, History, Biography, Drama]","[SE, FR, DK, US, RU, GB, BG, BR, DE, PL, HU, F...","[XX, DE, BG, JA]",Madame DuBarry,MADAMED
794,tt0002445,1.899,[Società Italiana Cines],488.0,6.078484,1913.0,120.0,"[History, Drama]","[FR, DK, GB, US, RU, IT, NL, HU, JP]","[LA, IT, JA]",Quo Vadis?,QUOVADI
800,tt0002461,1.051,"[M.B. Dudley Amusement Co., Le Film d'Art, Ste...",338.0,5.679290,1912.0,55.0,"[History, Drama, Adventure]","[GB, FR, US, RU, XWW, HU, JP]","[EN, XX, JA]",The Life and Death of King Richard III,THELIFE
...,...,...,...,...,...,...,...,...,...,...,...,...
800948,tt9907782,22.872,"[Piste Rouge, LD Entertainment]",19068.0,6.206204,2021.0,111.0,"[Mystery, Horror, Fantasy]","[EC, US, PH, IL, TW, JP, RO, GR, BR, DE, TR, A...","[FR, HI, TR, EN, JA]",The Cursed,THECURS
800950,tt9908390,7.731,"[Pathé, TF1 Films Production, Monkey Pack Film...",1627.0,5.481817,2020.0,95.0,[Comedy],"[RS, UA, FR, PT, RU, IT, CA, HR, GR, BE, TR, D...","[EN, TR, FR, JA]",The Lion,THELION
801039,tt9911196,3.420,"[Johan Nijenhuis & Co, Omroep MAX]",3418.0,7.403628,2020.0,103.0,"[Comedy, Drama]","[UA, FR, GB, US, NL, XWW, ES, HK]","[EN, NL, YUE]",The Marriage Escape,THEMARR
801091,tt9914972,1.418,[Third Man Films],319.0,7.107837,2021.0,96.0,[Documentary],"[US, PH, JP, IS, GR, BR, AU, SG, SE, IN, KR, N...","[FR, EN, SN, JA]",Blind Ambition,BLINDAM


In [136]:
df_filtered = df_film[~df_film['tconst'].isin(df['tconst'])]

In [138]:
df_filtered[['Titre', 'Description', 'Realisateur', 'Acteurs']] = df_filtered['tconst'].apply(lambda id: pd.Series(web(id)))

C:\Users\lepor\AppData\Local\Temp\ipykernel_5684\3770622716.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered[['Titre', 'Description', 'Realisateur', 'Acteurs']] = df_filtered['tconst'].apply(lambda id: pd.Series(web(id)))
C:\Users\lepor\AppData\Local\Temp\ipykernel_5684\3770622716.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered[['Titre', 'Description', 'Realisateur', 'Acteurs']] = df_filtered['tconst'].apply(lambda id: pd.Series(web(id)))
C:\Users\lepor\AppData\Local\Temp\ipyk

In [139]:
df = pd.concat([df, df_filtered], ignore_index=True)

In [125]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41706 entries, 0 to 41705
Data columns (total 15 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   tconst                     41706 non-null  object 
 1   Titre                      41657 non-null  object 
 2   Date                       41706 non-null  float64
 3   Duree                      41706 non-null  float64
 4   Genres                     41706 non-null  object 
 5   Region                     41706 non-null  object 
 6   Language                   41706 non-null  object 
 7   popularity                 41706 non-null  float64
 8   moyenne_vote               41706 non-null  float64
 9   nb_de_votes                41706 non-null  float64
 10  production_companies_name  41706 non-null  object 
 11  Saga                       41706 non-null  object 
 12  Description                41535 non-null  object 
 13  Realisateur                41657 non-null  obj

In [184]:
df_film = df_film.dropna(subset=['Titre'])

In [186]:
df_film.info()


<class 'pandas.core.frame.DataFrame'>
Index: 13706 entries, 27898 to 779629
Data columns (total 15 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   tconst                     13706 non-null  object 
 1   popularity                 13706 non-null  float64
 2   production_companies_name  13706 non-null  object 
 3   nb_de_votes                13706 non-null  float64
 4   moyenne_vote               13706 non-null  float64
 5   Date                       13706 non-null  float64
 6   Duree                      13706 non-null  float64
 7   Genres                     13706 non-null  object 
 8   Region                     13706 non-null  object 
 9   Language                   13706 non-null  object 
 10  Titre                      13706 non-null  object 
 11  Saga                       13706 non-null  object 
 12  Description                13706 non-null  object 
 13  Realisateur                13706 non-null  obj

In [188]:
ordre = [
    "tconst", "Titre", "Date", "Duree", "Genres", 
    "Region", "Language", "popularity", "moyenne_vote", 
    "nb_de_votes", "production_companies_name",
    'Saga', 'Description', 
    "Realisateur", "Acteurs"
]
df_film = df_film[ordre]

In [194]:
df_film[df_film['Titre'].str.contains('Star Wars')]

,tconst,Titre,Date,Duree,Genres,Region,Language,popularity,moyenne_vote,nb_de_votes,production_companies_name,Saga,Description,Realisateur,Acteurs
62393,tt0076759,Star Wars: Épisode IV - Un nouvel espoir,1977.0,121.0,"[Action, Fantasy, Adventure, Science Fiction]","[IL, DE, TR, CL, SI, UA, BG, AU, TH, NL, IS, U...","[HI, TR, SK, CS, FR, BG, CMN, SV, JA, FA, EN, ...",90.988,8.595133,1496579.0,"[20th Century Fox, Lucasfilm Ltd.]",STARWAR,"Luke Skywalker se joint à un chevalier Jedi, u...",George Lucas,"[Mark Hamill, Harrison Ford, Carrie Fisher, Al..."
95427,tt0120915,"Star Wars, épisode I : La Menace fantôme",1999.0,136.0,"[Action, Fantasy, Adventure, Science Fiction]","[IL, DE, TR, LV, HK, SI, UA, BG, TH, AU, AT, N...","[HI, TR, FR, BG, SV, CMN, FA, JA, EN, HE, ES, RU]",41.022,6.500000,888811.0,[Lucasfilm Ltd.],STARWAR,Deux chevaliers Jedi échappent à un blocus hos...,George Lucas,"[Ewan McGregor, Liam Neeson, Natalie Portman, ..."
95858,tt0121765,"Star Wars, épisode II : L'Attaque des clones",2002.0,142.0,"[Action, Fantasy, Adventure, Science Fiction]","[IL, DE, TR, LV, HK, SI, UA, BG, TH, AU, AT, N...","[HI, TR, FR, BG, SV, CMN, FA, JA, EN, ES, CA, RU]",36.750,6.599226,786408.0,[Lucasfilm Ltd.],STARWAR,"Dix ans après sa rencontre originale, Anakin S...",George Lucas,"[Hayden Christensen, Natalie Portman, Ewan McG..."
95859,tt0121766,"Star Wars, épisode III : La Revanche des Sith",2005.0,140.0,"[Action, Fantasy, Adventure, Science Fiction]","[IL, DE, TR, LV, HK, SI, UA, BG, TH, AT, AU, N...","[MS, HI, TR, FR, BG, SV, CMN, JA, EN, HE, ES, ...",39.515,7.597227,875184.0,[Lucasfilm Ltd.],STARWAR,Dans la troisième année de la Guerre des Clone...,George Lucas,"[Hayden Christensen, Natalie Portman, Ewan McG..."
317321,tt1185834,Star Wars: The Clone Wars,2008.0,98.0,"[Action, Animation, Adventure, Science Fiction]","[DE, TR, LV, SI, BG, AU, NL, US, TW, ES, KZ, J...","[HI, TR, FR, BG, JA, EN, RU]",26.473,5.904556,76608.0,"[Warner Bros. Pictures, Lucasfilm Animation, L...",STARWAR,Après la victoire de la République sur Christo...,Dave Filoni,"[Matt Lanter, Nika Futterman, Tom Kane, Ashley..."
638928,tt3748528,Rogue One: A Star Wars story,2016.0,133.0,"[Action, Adventure, Science Fiction, Sci-Fi]","[IL, DE, TR, LV, HK, CL, SI, UA, BG, TH, AT, A...","[HI, TR, FR, YUE, BG, CMN, KK, JA, EN, HE, ES,...",32.047,7.794121,715003.0,[Lucasfilm Ltd.],ROGUEON,La fille d'un scientifique impérial se joint à...,Gareth Edwards,"[Felicity Jones, Diego Luna, Alan Tudyk, Donni..."
640098,tt3778644,Solo: A Star Wars Story,2018.0,135.0,"[Action, Adventure, Science Fiction, Sci-Fi]","[IL, DE, TR, LV, HK, CL, SI, UA, BG, TH, AT, A...","[HI, TR, FR, BG, CMN, JA, EN, HE, ES]",40.890,6.894173,395098.0,"[Imagine Entertainment, Walt Disney Pictures, ...",SOLO:A,"Au cours d'une aventure dans la pègre, Han Sol...",Ron Howard,"[Alden Ehrenreich, Woody Harrelson, Emilia Cla..."


In [140]:
df.to_csv('df_film.csv.gz', index=False, compression='gzip')

In [36]:
df

,tconst,Titre,Date,Duree,Genres,Region,Language,popularity,moyenne_vote,nb_de_votes,production_companies_name,Saga,Description,Realisateur,Acteurs
0,tt0035423,Kate et Léopold,2001.0,118.0,"['Fantasy', 'Comedy', 'Romance']","['DE', 'TR', 'SI', 'UA', 'BG', 'AU', 'AT', 'US...","['TR', 'FR', 'IT', 'BG', 'CMN', 'HR', 'JA', 'E...",15.770,6.399042,91709.0,"['Konrad Pictures', 'Miramax']",KATE&L,Un duc anglais de 1876 se retrouve par inadver...,James Mangold,"['Meg Ryan', 'Hugh Jackman', 'Liev Schreiber',..."
1,tt0041719,Orphée,1950.0,112.0,"['Fantasy', 'Romance', 'Drama']","['TR', 'UA', 'BG', 'AU', 'AT', 'US', 'TW', 'ES...","['TR', 'FR', 'BG', 'SV', 'JA', 'EN', 'RU']",6.808,7.895434,13915.0,"['Films du Palais Royal', 'Andre Paulve Film']",ORPHEUS,Un poète fasciné par la Mort suit sa femme mal...,Jean Cocteau,"['Jean Marais', 'François Périer', 'María Casa..."
2,tt0041931,Stromboli,1950.0,81.0,['Drama'],"['TR', 'HK', 'BG', 'US', 'ES', 'DK', 'JP', 'GB...","['DE', 'TR', 'IT', 'FR', 'BG', 'CMN', 'JA', 'E...",8.494,7.195787,8368.0,"['Berit Films', 'RKO Radio Pictures']",STROMBOL,"Karen, une jeune femme de la Baltique, épouse ...",Roberto Rossellini,"['Ingrid Bergman', 'Mario Vitale', 'Renzo Cesa..."
3,tt0042192,Ève...,1950.0,138.0,['Drama'],"['IL', 'DE', 'TR', 'HK', 'SI', 'UA', 'BG', 'AU...","['MI', 'HI', 'TR', 'QBN', 'FR', 'BG', 'AR', 'S...",18.707,8.199517,142629.0,['20th Century Fox'],ALLABOU,Une ingénue se glisse dans la compagnie d'une ...,Joseph L. Mankiewicz,"['Bette Davis', 'Anne Baxter', 'George Sanders..."
4,tt0042200,"Annie, la reine du cirque",1950.0,107.0,"['Comedy', 'Action', 'Musical', 'Romance', 'We...","['TR', 'BG', 'AU', 'AT', 'US', 'ES', 'DK', 'JP...","['TR', 'QBN', 'FR', 'BG', 'SV', 'JA', 'EN', 'E...",6.471,6.900000,5426.0,['Metro-Goldwyn-Mayer'],ANNIEGE,The story of the great sharpshooter Annie Oakl...,George Sidney,"['Betty Hutton', 'Howard Keel', 'Louis Calhern..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60514,tt9896252,Killing Patient Zero,2019.0,100.0,"['History', 'Documentary']","['CA', 'GB', 'DE', 'AU', 'US', 'PL', 'GR']","['FR', 'EN']",2.290,7.484928,345.0,['Fadoo Productions'],KILLING,Gaetan Dugas was openly gay. In early 1980s he...,Laurie Lynd,"['Richard Berkowitz', 'Richard Bisson', 'Anita..."
60515,tt9903716,Jessie,2019.0,106.0,"['Horror', 'Drama', 'Thriller']","['CA', 'GB', 'IN', 'US', 'SG']","['TA', 'TE', 'EN', 'FR', 'HI']",1.641,6.688531,497.0,[],JESSIE,"Set in an abandoned house, the film follows a ...",Aswani Kumar V.,"['Ashima Narwal', 'Sritha Chandana', 'Atul Kul..."
60516,tt9904802,Enemy Lines,2020.0,92.0,"['Action', 'Drama', 'War']","['CA', 'GB', 'PT', 'AU', 'DE', 'US', 'NL', 'PL...","['JA', 'DE', 'PL', 'EN', 'RU', 'FR']",5.523,4.611144,2010.0,"['Principal Film Finance', 'Gaia Media', 'Good...",ENEMYLI,Sous l'occupation en Pologne pendant la Second...,Anders Banke,"['Ed Westwick', 'John Hannah', 'Tom Wisdom', '..."
60517,tt9907782,Eight for Silver,2021.0,111.0,"['Mystery', 'Fantasy', 'Horror']","['CA', 'EG', 'AU', 'PL', 'AE', 'TR', 'ES', 'RU...","['JA', 'EN', 'TR', 'FR', 'HI']",22.872,6.206204,19068.0,"['Piste Rouge', 'LD Entertainment']",THECURS,"Dans la France rurale du 19ème siècle, une men...",Sean Ellis,"['Boyd Holbrook', 'Kelly Reilly', 'Alistair Pe..."
